# file-IO in Python and Exceptions

---

In this notebook I want to introduce the file IO in Python and unusually address another topic, exceptions or exception handling.

---

## 1. File-IO in Python

For reading text files with some data in table form we already know `np.loadtxt` from the `numpy` module. Basically Python itself provides a nice set of IO operations.

As a demonstration I've created a simple text file `data/simple.txt`:

In [ ]:
!cat data/simple.txt

The task is to read this file in Python:

In [ ]:
# reading a text file #1

f = open('data/simple.txt', 'r')    # open the file for _r_reading

lines = f.readlines()               # read all lines into the memory

print(type(lines))
print(lines)


f.close()                           # all files need to be closed! 

As you can see all lines of the text file will be read together and a list of strings will be returned. Each line has at the end the `\n` character and one line has some additional indent:

In [ ]:
for line in lines:
    line = line.strip()
    print(line)

This code fragment shows how to get rid of the extra characters. If you need the leading trails, you can simply use `.rstrip()` instead of `.strip()` (please read the documentation).

Since usually one works with simple lines, one can also use a `for`-loop over the file object `f`:

In [ ]:
# reading a text file #2

f = open('data/simple.txt', 'r')  # open the file for _r_reading

for line in f:                    # work on all lines
    line = line.strip()           # get rid of trailing characters
    print(line)
f.close()                         # all files need to be closed!

## 2. Details of file-IO

The syntax of opening a file is:

```
f = open(<pathtoafile>, mode)

# file operations

f.close()
```

mode can be one of:
```
 r : read a file
 w : write to an existing file (file will be truncated at the beginning)
 x : create and write to a file
 a : append to a file
 + : open a disk file for reading and writing
```

`f` is always a file object which will be used to access the IO space.

For every `open` command a corresponding `close` command has to follow!

## 3. Saving files

Typically the commands are similar:

In [ ]:
# writing a text file

text = ['first line', 'second line']

f = open('output.txt', 'w') # open a file for writing

for line in text:
    f.write(f'{line}\n')    # using f-strings
    #f.write(line+'\n')      # raw command
    #print(line, file=f)    # alternative command

f.close()                   # all files need to be closed

You need to keep in mind to add a `\n` character for each line. Also don't forget to close the file with `.close()`, otherwise data may be lost or gets corrupted!

## 4. Binary files

Reading and writing binary files are also supported, where you need to open the files with an additional `b`. Reading will be done with `.read()` and writing with `.write`. You have to take care of the format and correctness of the data.

Usually you should use predefined modules to read special binary files, e.g. images, data cubes etc.

---

## 5. Problems with file-IO

Usually all these commands are working without any problems, but here and there some problems will occur:

 * you want to open a file for reading which does not exist
 * you want to overwrite an existing file (mode=`x`)
 * you don't have the rights to open a file for reading/writing
 * the disk will be full during writing
 * the destination vanishes somehow
 * ...
 
What happens?

In [ ]:
# open an non existing file for reading

f = open('data/simple2.txt', 'r')
lines = f.readlines()
f.close()

print('Another command which should be executed afterwards!')

This is expected and okay for some test programs. 

**What happend?**

During the file operation an error occurs and the program simply aborts directly. As said, for test or smaller programs it is okay, you can check for the error and restart the program with the correct file name.

If you e.g. do a long simulation and shortly before the simulation is finished a write to a log file produces an error, all of your results are gone ... :-(

---

## 6. Catching errors or exceptions

One solution is to catch these kind of errors to leave your program running and decide what is best to do:

In [ ]:
# open an non existing file for reading

try:
    f = open('data/simple2.txt', 'r')
    lines = f.readlines()
    f.close()
except:
    print('Some error occurred!')

print('Another command which should be executed afterwards!')

In this case all `errors` which can cause an so called `exception` within the `try` block can be catched, so that the program can decide, how an error will be handled.

Of course the catching can be explicitly defined for a certain exception:

In [ ]:
# open an non existing file for reading

try:
    f = open('data/simple2.txt', 'r')
    lines = f.readlines()
    f.close()
except IOError:
    print('Some IO related error occurred!')

print('Another command which should be executed afterwards!')

Exceptions have usually some arguments or features:

In [ ]:
# open an non existing file for reading

try:
    f = open('data/simple2.txt', 'r')
    lines = f.readlines()
    f.close()
except IOError as err:
    print(type(err))
    print(err)
    print(err.args)
    print(f'Some IO error occurred "{err}"!')    # my suggestion!

print('Another command which should be executed afterwards!')

---

## 7. Basic exceptions

Python provides a large variety of predefined exceptions:

  * `IOError`:  everything around IO operations
  * `ValueError`: used for failing conversions
  * `TypeError`: failing types
  * `IndexError`: failing indices (lists, tuples, numpy-arrays)
  * `KeyError`: failing keys  (dicionaries)
  * `NameError`: using of not defined variables
  * `ZeroDivisionError`:  division with 0
  * `KeyboardError` : if you hit Ctrl-C during script execution
  * ...

In [ ]:
int('hallo')   # value error

In [ ]:
s = 'Hallo'
s + 1         # type error

In [ ]:
a = [1,2]
a[100]     # index error

In [ ]:
d = { '1': 1 }
d['2']         # key error

In [ ]:
spam*3    # name error

Not all exceptions need to be catched and in generall indicating errors which can be in most cases solved by additional tests!

In [ ]:
import os

# open an non existing file for reading

filename = 'data/simple2.txt'

if os.access(filename, os.R_OK):
    try:
        f = open(filename, 'r')
        lines = f.readlines()
        print(lines)
        f.close()
    except IOError as err:
        print(f'Some IO error occurred "{err}"!')    # my suggestion!
else: # os.access
    print(f'Can\'t access "{filename}"!')

print('Another command which should be executed afterwards!')

---

## 8. Try-except-else-finally

To catch exceptions you can use these syntax form:

```
try:
    # block of commands
except <exception1> [as <var1>]:
    # block executed if exception1 occurs
except <exception2> [as <var2>]:
    # block executed if exception2 occurs
except ...  # more exceptions
else:
    # block executed if no exception happed
finally:
    # will be executed always!
```

### else

The role of `else` is not really intuitive, in our example it means, that we can devide into code which is checked for exceptions and code which is executed if no exception happened:

In [ ]:
# open an non existing file for reading

try:
    f = open('data/simple2.txt', 'r')
except IOError:
    print('Some IO related error occurred!')
else:
    lines = f.readlines()
    print(lines)
    f.close()

print('Another command which should be executed afterwards!')

**Note:** An exception can happen during the execution of the `else` block which will not be catched!

### finally

`finally` plays a different role than `else`. The finally block will always be executed, which is useful e.g. during file operations, where the file can be properly closed before the program stops. This prevents some data loss:

In [ ]:
# open an non existing file for reading

try:
    f = open('data/simple2.txt', 'r')
except IOError:
    print('Some IO related error occurred!')
else:
    lines = f.readlines()
    print(lines)
finally:
    print('finally')
    f.close()

print('Another command which should be executed afterwards!')

**Note:** Personally I avoid the `else` part and put all the commands inside the `try` block, since the complete block is under control!

---

## 9. Exceptions somewhere and nested exception catching

Whenenver an exception occurs, event in deeper function calls the program will then continue in the except part. This means also, that not handled exceptions can be handled in other functions:

In [ ]:
def faulty(x,y):
    res=x/y
    return res
    
try:
    faulty(1,0)
except ZeroDivisionError:
    print('zero division')

or:

In [ ]:
def faulty(x,y):
    try:
        res=x/y
    except ZeroDivisionError:
        print('zero division')
    return res
    
try:
    faulty(1,0)
except UnboundLocalError as err:
    print(err)
    

---

## 10. file-IO with `with`

For generating more compact code for file-IO operations, you can use the `with` statement:

In [ ]:
with open('data/simple.txt','r') as f:
    lines = f.readlines()
    print(lines)

In this case `f.close()` will be executed after the block is successfully executed, even if in the block an exception happens. This will avoid data loss during file write operations.

Anyway, problems need to be catched:

In [ ]:
try:
    with open('data/simple.txt','r') as f:
        lines = f.readlines()
        print(lines)
except IOError:
    print('Some IO related error occurred!')

---

## 11. Raising own exception

Sometimes it is necessary to throw an exception by your self. At the moment any other code may be better, but e.g. in Python `class` definitions, this is the only way to indicate an error:

In [ ]:
def test_function(x):
    """
    test_function prooves if x is in [0,1]
    if x is out of the interval, a ValueError will be thrown
    
    Example:
    try:
        print(test_function(-1))
    except ValueError as err:
        print(err)
    """
    if (x > 1) or (x<0):
        raise ValueError(f'variable x={x} out of limits')
    return x

# main program
print(test_function(0.5))

try:
    print(test_function(-1))
except ValueError as err:
    print(err)

**Note:** Choose an Exceptions which is close to the reason, you want to have. A message is always useful! Document you function, so that anybody knows, that this function will throw an exception!

---

## 12. Debugging

Under some circumstances, e.g. in complex python scripts, it is necessary put a `try` command around a large block of commands for debugging and check the influence of errors to the program. At the same time one is interested in where the problem happened. Python provides a nice debugging tool for it:

In [ ]:
import traceback

try:
    print(1/0)
except:
    print('error')
    tcbk = traceback.format_exc()

    print(tcbk)

print('Normal execution')

---

## 13. Logging

For larger python projects, it is useful, to document some program steps or also address warning or even errors of exceptions. Python provides a useful module `logging` which can be used for the creations of different logging situations:

In [5]:
import logging

logging.info('Useful informations!')
logging.debug('Some debug notes!')
logging.warning('Some warnings.')
logging.error('Some errors.')
logging.critical('Critical errors.')

ERROR:root:Some errors.
CRITICAL:root:Critical errors.


In jupyter notebooks, the `logging`-module is not working correctly, but it is working nicely in python `scripts`:

In [6]:
!cat logging_demo.py

import logging

#logging.basicConfig(level=logging.DEBUG)
#logging.basicConfig(filename='demo.log', format='%(asctime)s %(message)s', level=logging.DEBUG )

logging.info('Useful informations!')
logging.debug('Some debug notes!')
logging.warning('Some warnings.')
logging.error('Some errors.')
logging.critical('Critical errors.')
